In [1]:
import os 
import pandas as pd 

In [2]:
os.listdir("../data/national_march_2020")

['MANIFEST.TXT', 'MUP213_NATL_qre_archive.pdf', 'MUP213_NATL_archive.tab']

In [3]:
nat_2020 = pd.read_csv("../data/national_march_2020/MUP213_NATL_archive.tab", sep = "\t")

In [4]:
nat_2020.rename({
    'Q2': 'approve_trump',
    'Q11': 'likely_to_vote', 
    'Q12': 'vote_choice', 
    'Q12B': 'vote_choice_undecided', 
    ''
})

SyntaxError: ':' expected after dictionary key (2271976407.py, line 6)

In [5]:
nat_2020

,RESPID,PHTYPE,Q1,Q2,Q3,Q4,Q5,Q6A,Q6B,Q6C,...,QD4,QD5,QD5A,QD6,QD7,QD8,QD9,QD10,QD11,FINALWGT
0,11,2,9,1,20,2,2,1,9,1,...,5,53,NaN,2,2,1,4,1,24.0,0.879980
1,14,2,1,1,29,2,2,1,1,1,...,5,28,NaN,1,2,1,2,1,34.0,1.089499
2,19,2,1,1,28,3,1,1,1,1,...,7,75,NaN,2,2,1,3,1,37.0,0.520273
3,30,2,2,2,20,1,3,1,2,1,...,7,63,NaN,2,2,1,2,2,37.0,0.573839
4,43,2,9,9,20,1,2,2,3,3,...,7,47,NaN,1,2,5,9,2,13.0,0.972234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,3746,1,2,2,10,1,3,3,2,1,...,7,51,NaN,1,2,1,4,1,26.0,0.469501
847,3752,1,2,2,20,2,4,3,2,1,...,5,21,NaN,2,2,1,4,2,24.0,1.207679
848,3754,2,1,1,30,4,9,1,1,1,...,6,99,9.0,2,2,1,2,2,12.0,0.906539
849,3762,1,2,2,20,1,2,1,3,1,...,7,49,NaN,2,2,2,1,2,22.0,0.665262
